# A simple CNN for the edge lover task

In this notebook you train a very simple CNN with only 1 kernel to distinguish between images containing vertical and images containing horizontal stripes. To check what pattern is recognized by the learned kernel you will visualize the weights of the kernel as an image. You will see that the CNN learns a useful kernel (either a vertical or horiziontal bar). You can experiment with the code to check the influence of the kernel size, the activation function and the pooling method on the result.  


**Dataset:** You work with an artficially generated dataset of greyscale images (50x50 pixel) with 10 vertical or horizontal bars. We want to classify them into whether an art lover, who only loves vertical strips, will like the image (y = 0) or not like the image (y = 1).  

The idea of the notebook is that you try to understand the provided code by running it, checking the output and playing with it by slightly changing the code and rerunning it.  

**Content:**
* definig and generating the dataset X_train and X_val
* visualize samples of the generated images
* use keras to train a CNN with only one kernel (5x5 pixel)
* visualize the weights of the learned kernel and interpret if it is useful
* repeat the last two steps to check if the learned kernel is always the same



### Imports

In the next cell, we load all the required libraries.

In [ ]:
# load required libraries:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')

import tensorflow.keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Convolution2D, MaxPooling2D, Flatten , Activation
from tensorflow.keras.utils import to_categorical

### Defining functions to generate images

Here we define the function to genere images with vertical and horizontal bars, the arguments of the functions are the size of the image and the number of bars you want to have. The bars are at random positions in the image with a random length. The image is black and white, meaning we have only two values for the pixels, 0 for black and 255 for white.

In [ ]:
#define function to generate image with shape (size, size, 1) with stripes
def generate_image_with_bars(size, bar_nr, vertical = True):
    img = np.zeros((size,size,1), dtype="uint8")
    for i in range(0,bar_nr):
        x,y = np.random.randint(0,size,2)
        l  = int(np.random.randint(y,size,1)[0])
        if (vertical):
            img[y:l,x,0]=255
        else:
            img[x,y:l,0]=255
    return img

Let's have a look at the generated images. We choose a size of 50x50 pixels and set the number of bars in the image to 10.

In [ ]:
# have a look on two generated images
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
img=generate_image_with_bars(50,10, vertical=True)
plt.imshow(img[:,:,0],cmap='gray')
plt.subplot(1,2,2)
img=generate_image_with_bars(50,10, vertical=False)
plt.imshow(img[:,:,0],cmap='gray')
plt.show()

### Make a train and validation dataset of images with vertical and horizontal images
Now, let's make a train dataset *X_train* with 1000 images (500 images with vertical and 500 images with horizontal bars). We normalize the images values to be between 0 and 1 by dividing all values with 255. We create a secont dataste *X_val* with exactly the same properties to validate the training of the CNN.

In [ ]:
pixel=50  # define height and width of images
num_images_train = 1000 #Number of training examples (divisible by 2)
num_images_val = 1000 #Number of training examples (divisible by 2)

# generate training data with vertical edges
X_train =np.zeros((num_images_train,pixel,pixel,1))
for i in range(0, num_images_train//2):
    X_train[i]=generate_image_with_bars(pixel,10)
# ... with horizontal
for i in range(num_images_train//2, num_images_train):
    X_train[i]=generate_image_with_bars(pixel,10, vertical=False)

# generate validation data with vertical edges
X_val =np.zeros((num_images_train,pixel,pixel,1))
for i in range(0, num_images_train//2):
    X_val[i]=generate_image_with_bars(pixel,10)
# ... with horizontal
for i in range(num_images_train//2, num_images_train):
    X_val[i]=generate_image_with_bars(pixel,10, vertical=False)

In [ ]:
# normalize the data to be between 0 and 1
X_train=X_train/255
X_val=X_val/255

print(X_train.shape)
print(X_val.shape)

Here we make the labels for the art lover, 0 means he likes the image (vertical bars) and 1 means that he doesn't like it (horizontal stripes). We one hot encode the labels because we want to use two outputs in our network.

In [ ]:
# create class labels
y = np.array([[0],[1]])
Y_train = np.repeat(y, num_images_train //2)
Y_val = np.repeat(y, num_images_train //2)

# one-hot-encoding
Y_train = to_categorical(Y_train,2)
Y_val = to_categorical(Y_val,2)

## Defining the CNN

Here we define the CNN:

- we use only one kernel with a size of 5x5 pixels  
- then we apply a linar activation function  
- the maxpooling layer takes the maximum of the whole activation map to predict the probability (output layer with softmax) if the art lover will like the image

As loss we use the categorical_crossentropy and we train the model with a batchsize of 64 images per update.


In [ ]:
model = Sequential()

model.add(Convolution2D(1,(5,5),padding='same',input_shape=(pixel,pixel,1)))
model.add(Activation('linear'))

# take the max over all values in the activation map
model.add(MaxPooling2D(pool_size=(pixel,pixel)))
model.add(Flatten())
model.add(Dense(2))
model.add(Activation('softmax'))

# compile model and initialize weights
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [ ]:
# let's summarize the CNN architectures along with the number of model weights
model.summary()


In [ ]:
# train the model
history=model.fit(X_train, Y_train,
                  validation_data=(X_val,Y_val),
                  batch_size=64,
                  epochs=150,
                  verbose=1)

In [ ]:
# plot the development of the accuracy and loss during training
plt.figure(figsize=(12,4))
plt.subplot(1,2,(1))
plt.plot(history.history['accuracy'],linestyle='-.')
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='lower right')
plt.subplot(1,2,(2))
plt.plot(history.history['loss'],linestyle='-.')
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper right');

### Visualize the learned kernel and experiment with the code

You see that the CNN performs very good at this task (100% accuracy). We can check which pattern is recognized by the **learned kernel** and see if you think that this is helpful to distinguish between images with horizontal and vertical edges.

Below you can see the original image, the image after the convolution operation with the learned kernel and the maximum value from the maxpooling operation. Note that the maxpooling has the same size as the convolved image so there is just one value as output.

The probability (or more exactly likelihood) assigned by the model for each sample to belong to class "horizontal" or "vertical" is displayed P(y="horizontal"|x),P(y="vertical"|x) . The -log(P(y="horizontal"|x)) or -log(P(y="vertical"|x))  (the NLL : NegativeLogLikelihoods) which are used in the loss function 
are displayed aswell. 



Move the sliders to inspect different pictures from the validation set and their predictions


#### Extra

try to lower (or increase) the training epochs and see how the -log(P(y|x)) change.

In [ ]:
## Do not worry about this cell, just move the sliders.
import scipy.signal
from skimage.measure import block_reduce  # For max pooling
import ipywidgets as widgets

# Kernel from model
plt.figure(figsize=(10, 3))
plt.subplot(1, 2, 1)
plt.imshow(np.random.rand(25).reshape(5, 5),"gray") ,plt.title('Randomly initalized weights')
plt.subplot(1, 2, 2)
conv_filter=np.squeeze(model.get_weights()[0], axis=2)
plt.imshow(conv_filter[:,:,0],"gray"),plt.title('Learned Kernel (weights) , by model'),plt.show();
print("\n---------Move the sliders to inspect different vertical and horizontal images from the valset and their predictions:------------------\n")

def scale_convolution_map(conv_map, min_val=-3, max_val=3):
    clipped_conv_map = np.clip(conv_map, min_val, max_val)
    scaled_conv_map = (clipped_conv_map - min_val) / (max_val - min_val)
    return scaled_conv_map

def plot_conv(img):
  convolved_image = scipy.signal.convolve2d(img.squeeze(), conv_filter.squeeze(), mode='same')
  scaled_conv_image = scale_convolution_map(convolved_image + model.get_weights()[1])
  max_pooled_image = block_reduce(convolved_image + model.get_weights()[1], block_size=(50, 50), func=np.max)
  scaled_max_pooled_image = scale_convolution_map(max_pooled_image)
  
  plt.figure(figsize=(20, 4))  # Adjust the figure size as needed
  plt.subplot(1, 6, 1)
  plt.imshow(img, "gray", vmin=0, vmax=1),plt.title('Original Image')
  plt.subplot(1, 6, 2)
  plt.imshow(scaled_conv_image, "gray", vmin=0, vmax=1),plt.title('Convolved Image')
  plt.subplot(1, 6, 3),plt.imshow(scaled_max_pooled_image, "gray", vmin=0, vmax=1)
  plt.title(f'Max Pooled = {max_pooled_image[0][0]:.2f}'),plt.xticks([]), plt.yticks([])
  plt.subplot(1, 6, 4),plt.axis('off')
  pred = model.predict(img.reshape(1, 50, 50, 1), verbose=0)
  text_info = f'''
  P(y=vertical|x): {pred[0][0]:.4f}
  P(y=horizontal|x): {pred[0][1]:.4f}
                  
                  
   -log(P(y=vertical|x)): {-np.log(pred[0][0]):.4f}
   -log(P(y=horizontal|x)): {-np.log(pred[0][1]):.4f}
    '''
  plt.text(0, 0.5, text_info, ha='left', va='center')
  plt.subplot(1, 6, 5)
  x_values = np.linspace(0.001, 1.1, 500)
  plt.plot(x_values, -np.log(x_values), label='-log(P(y|x))')
  plt.ylim(-0.5, 6),plt.xlim(-0.1, 1.1),plt.xlabel('P(y|x)')
  plt.plot(pred[0][0], -np.log(pred[0][0]), 'bo', label='-log(P(y=vertical|x))')
  plt.plot(pred[0][1], -np.log(pred[0][1]), 'ro', label='-log(P(y=horizontal|x))')
  plt.legend(),plt.grid(True), plt.tight_layout(),plt.show();

def inspect_preds(horizontal,vertical):
  plot_conv(X_val[horizontal,:,:,0])
  plot_conv(X_val[vertical,:,:,0])

horizontal_slider = widgets.IntSlider(min=0, max=num_images_val//2-1, step=1, value=0, description='vertical ')
vertical_slider = widgets.IntSlider(min=num_images_val//2, max=num_images_val-1, step=1, value=0, description='horizontal')
widgets.interact(inspect_preds, horizontal=horizontal_slider, vertical=vertical_slider);

### Repeat the training and experiment with the kernelsize and activation function.

**Exercise**:
- Repeat the compiling and training, beginning from the cell:

```
model = Sequential()
       
       ...
       
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
```

for several times and check if the CNN always learns the same kernel.  

- You can experiment with the code and check what happens if you use another kernel size, activation function (relu instead of linear ) or pooling method AveragePooling instead of MaxPooling.  Try to make a prediction on the performance before doing the experiment.


